# 6. Comparaison des Performances des Modèles

## 6.1. Introduction

Ce notebook finalise le projet en comparant les performances de tous les modèles entraînés sur un jeu de données de test commun. Cela nous permet d'évaluer objectivement quelle approche est la plus efficace pour cette tâche de classification de sentiments.

**Modèles Comparés :**
- Régression Logistique
- Support Vector Machine (SVM)
- BERT (fine-tuné avec LoRA)
- XLNet (fine-tuné)

## 6.2. Installation et Importations

In [ ]:
!pip install torch transformers scikit-learn pandas joblib matplotlib seaborn -q

In [ ]:
import torch
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import AutoTokenizer, BertForSequenceClassification, XLNetForSequenceClassification

## 6.3. Chargement de tous les modèles et pré-requis

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

# Charger les modèles classiques
lr_model = joblib.load('../models/logistic_regression/logreg_model.pkl')
lr_vectorizer = joblib.load('../models/logistic_regression/tfidf_vectorizer.pkl')
label_encoder = joblib.load('../models/logistic_regression/label_encoder.pkl')

svm_model = joblib.load('../models/svm/svm_model.pkl')
svm_vectorizer = joblib.load('../models/svm/tfidf_vectorizer.pkl')

# Charger les modèles Transformer
bert_model = BertForSequenceClassification.from_pretrained("../models/bert_lora").to(device)
bert_tokenizer = AutoTokenizer.from_pretrained("../models/bert_lora")

xlnet_model = XLNetForSequenceClassification.from_pretrained("../models/xlnet").to(device)
xlnet_tokenizer = AutoTokenizer.from_pretrained("../models/xlnet")

print("\nTous les modèles ont été chargés avec succès.")

## 6.4. Préparation du Jeu de Données de Test

In [ ]:
full_data = pd.read_csv('../data/balanced_subset.csv')
test_sample = full_data.sample(n=10000, random_state=42)
X_test_text = test_sample['cleaned_text'].dropna().tolist()
y_test_true_labels = test_sample.loc[test_sample['cleaned_text'].notna(), 'rating'].tolist()

## 6.5. Exécution des Prédictions

In [ ]:
def predict_transformer(texts, model, tokenizer, batch_size=32):
    predictions = []
    model.eval()
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
    return np.array(predictions)

# Prédictions des modèles classiques
lr_preds_labels = label_encoder.inverse_transform(lr_model.predict(lr_vectorizer.transform(X_test_text)))
svm_preds_labels = svm_model.predict(svm_vectorizer.transform(X_test_text))

# Prédictions des modèles Transformer
bert_preds_encoded = predict_transformer(X_test_text, bert_model, bert_tokenizer)
xlnet_preds_encoded = predict_transformer(X_test_text, xlnet_model, xlnet_tokenizer)

bert_preds_labels = label_encoder.inverse_transform(bert_preds_encoded)
xlnet_preds_labels = label_encoder.inverse_transform(xlnet_preds_encoded)

## 6.6. Rapports de Performance

In [ ]:
print("--- Performance de la Régression Logistique ---")
print(classification_report(y_test_true_labels, lr_preds_labels))

print("--- Performance du SVM ---")
print(classification_report(y_test_true_labels, svm_preds_labels))

print("--- Performance de BERT ---")
print(classification_report(y_test_true_labels, bert_preds_labels))

print("--- Performance de XLNet ---")
print(classification_report(y_test_true_labels, xlnet_preds_labels))

## 6.7. Visualisation de la Comparaison

In [ ]:
model_names = ['Logistic Regression', 'SVM', 'BERT', 'XLNet']
accuracies = [
    accuracy_score(y_test_true_labels, lr_preds_labels),
    accuracy_score(y_test_true_labels, svm_preds_labels),
    accuracy_score(y_test_true_labels, bert_preds_labels),
    accuracy_score(y_test_true_labels, xlnet_preds_labels)
]

plt.figure(figsize=(10, 6))
sns.barplot(x=model_names, y=accuracies, palette='viridis')
plt.title('Comparaison de la Précision des Modèles')
plt.ylabel('Précision (Accuracy)')
plt.ylim(0.8, 0.95) # Ajuster les limites pour une meilleure visualisation

for index, value in enumerate(accuracies):
    plt.text(index, value + 0.005, f'{value:.3f}', ha='center', fontsize=12)

plt.show()

### Matrices de Confusion

In [ ]:
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred, labels=label_encoder.classes_)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title(f'Matrice de Confusion - {model_name}')
    plt.xlabel('Prédit')
    plt.ylabel('Vrai')
    plt.show()

plot_confusion_matrix(y_test_true_labels, lr_preds_labels, 'Régression Logistique')
plot_confusion_matrix(y_test_true_labels, svm_preds_labels, 'SVM')
plot_confusion_matrix(y_test_true_labels, bert_preds_labels, 'BERT')
plot_confusion_matrix(y_test_true_labels, xlnet_preds_labels, 'XLNet')